In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed
import umap
import pandas as pd

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from scipy.io import loadmat
from pathlib2 import Path

In [ ]:
from avgn.utils.hparams import HParams
from avgn.dataset import DataSet

In [ ]:
from vocalseg.utils import butter_bandpass_filter, spectrogram, int16tofloat32, plot_spec
from vocalseg.dynamic_thresholding import dynamic_threshold_segmentation

In [ ]:
DATASET_ID = 'gibbon_morita_segmented'

In [ ]:
### segmentation parameters
n_fft=1024
hop_length_ms=5
win_length_ms=20
ref_level_db=20
pre=0.97
min_level_db=-70
min_level_db_floor = -50
db_delta = 5
silence_threshold = 0.01
min_silence_for_spec=0.05
max_vocal_for_spec=10,
min_syllable_length_s = 0.5
butter_min = 50
butter_max = 7999
spectral_range = [50, 7999]

In [ ]:
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir

In [ ]:
hparams = HParams(
    butter_lowcut = 50,
    butter_highcut = 7999,
mel_upper_edge_hertz=7999,)
# create a dataset object

dataset = DataSet(DATASET_ID, hparams = hparams)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  81 out of 128 | elapsed:    8.2s remaining:    4.8s


[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:    8.4s finished


In [ ]:
dataset.sample_json

OrderedDict([('datetime', '2017-09-12_07-00-02'),
             ('species', 'Hylobatidae'),
             ('common_name', 'Gibbon'),
             ('samplerate_hz', 16000),
             ('original_wav',
              '/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/gibbon_morita/gibbon_superregular_data/test_data/20170912-070002_01_ch.wav'),
             ('length_s', 1800.0),
             ('wav_loc',
              '/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/gibbon_morita/gibbon_superregular_data/test_data/20170912-070002_01_ch.wav'),
             ('indvs',
              OrderedDict([('UNK',
                            OrderedDict([('syllables',
                                          OrderedDict([('start_times',
                                                        [349.04,
                                                         390.04,
                                                         422.44,
                                                     

In [ ]:
import librosa
from vocalseg.utils import butter_bandpass_filter, spectrogram, int16tofloat32, plot_spec
from vocalseg.dynamic_thresholding import dynamic_threshold_segmentation
from PIL import Image
from avgn.umap import umap_reduce
import seaborn as sns

In [ ]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [ ]:
# for each wav, compute the spectrogram, break the spectrogram up into multiple segments, then project into PCA, then plot PCA projections over time

In [ ]:
indvs = np.array([list(i)[0] for i in dataset.json_indv])
np.unique(indvs)

array(['UNK'], dtype='<U3')

In [ ]:
indv = 'UNK' 
indv_keys = np.array(list(dataset.data_files.keys()))[indvs==indv]
silence_thresh_seconds = 25

In [ ]:
window_list = [200]
window_length_ms = window_list[0]
specs = []
key_windows = {i:{} for i in window_list}

In [ ]:
### for each key, get bouts with pauses > 10 seconds, save the windows from that bout, the key, the start and end time of the bout

In [ ]:
from itertools import islice

def window(seq, n=2, skip = 100):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [ ]:
windows_df = pd.DataFrame(columns = ['key', 'bout_number', 'bout_start_time', 'bout_end_time', 'bout_window', 'bout_spec'])

for key in tqdm(indv_keys):    
    # load data
    wf = dataset.data_files[key].data['wav_loc']
    data, rate = librosa.core.load(wf, sr=None)
    data = butter_bandpass_filter(data, 500, 7999, rate, order = 2)
    
    # get start and end times
    start_times = dataset.data_files[key].data['indvs']['UNK']['syllables']['start_times']
    end_times = dataset.data_files[key].data['indvs']['UNK']['syllables']['end_times']
    if len(start_times) < 20: 
        continue
    
    # get split times
    split_points = [False] + list(np.array(start_times[1:]) - (end_times[:-1]) > silence_thresh_seconds)
    split_point_locs = [0] + list(np.where(split_points)[0])
    if split_point_locs[-1] != len(split_points) -1:
        split_point_locs = split_point_locs + [len(split_points) -1]

    # get the spectrogram
    # segment
    results = dynamic_threshold_segmentation(
        data,
        rate,
        n_fft=n_fft,
        hop_length_ms=hop_length_ms,
        win_length_ms=win_length_ms,
        min_level_db_floor=min_level_db_floor,
        db_delta=db_delta,
        ref_level_db=ref_level_db,
        pre=pre,
        min_silence_for_spec=min_silence_for_spec,
        max_vocal_for_spec=max_vocal_for_spec,
        min_level_db=min_level_db,
        silence_threshold=silence_threshold,
        verbose=False,
        min_syllable_length_s=min_syllable_length_s,
        spectral_range=spectral_range,
    )
    spec = results['spec']
    spec = np.array(Image.fromarray(spec).resize((np.shape(spec)[1], 32), Image.ANTIALIAS))
    
    # get bouts
    for bi, (sp0, sp1) in enumerate(zip(split_point_locs[:-1], split_point_locs[1:])):
        spec_st = int(start_times[sp0] / (hop_length_ms/ 1000))
        spec_et = int(end_times[sp1-1] / (hop_length_ms/ 1000))
        bout_spec = spec[:, spec_st:spec_et]
        
        # get windows
        window_length_frames = int(window_length_ms / hop_length_ms)
        windows = [i for i in window(bout_spec.T, n=window_length_frames)]
        windows = np.reshape(windows, (len(windows), np.product(np.shape(windows)[1:])))
        
        windows_df.loc[len(windows_df)] = [key, bi, sp0, sp1, windows, bout_spec]

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/mnt/cube/tsainbur/Projects/github_repos/vocalization_segmentation/vocalseg/dynamic_thresholding.py:129: RuntimeWarning: invalid value encountered in sqrt
  vocal_envelope = np.max(spec, axis=0) * np.sqrt(np.mean(spec, axis=0))
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f933031c468>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f9330024eb8>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f9330024c50>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93300542b0>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignore

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93300584c0>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f9330058258>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93300584c0>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignore

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93103d05c8>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93103d0990>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f93103d0ba0>
Traceback (most recent call last):
  File "/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignore

In [ ]:
ensure_dir(DATA_DIR/'umap_windows' / DATASET_ID)
save_loc = DATA_DIR/'umap_windows' / DATASET_ID / ('gibbon_windows_' + str(window_length_ms)+ '.pickle')

In [ ]:
windows_df.to_pickle(save_loc)